<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EC%9E%A5%EA%B8%B0/%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keybert
!pip install kiwipiepy
!pip install google-cloud-vision
!pip install PyMuPDF
!pip install transformers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.2 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=6cce3c0259d31d9dd24d62e33964aa675d12d32fad73afc92a23b65fc4e798f8
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d17dc499acd9486809e302ff5c7569071c7fd0a20fa7feeebbbb509697d42f99
 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 38.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import fitz
import zipfile
import cv2
import io
import os
import torch
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel
from PIL import Image
from google.cloud import vision
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration

import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration, BartForCausalLM
# 모델과 토크나이저 로드
# model = BartForConditionalGeneration.from_pretrained('/content/kobart_summary')
summary_model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/KoBART-summarization/kobart_summary')
summary_tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')


### 키워드 명사로만 제공
from kiwipiepy import Kiwi
from transformers import BertModel
from keybert import KeyBERT

# 텍스트 전문 : full_text
# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()

# BERT 모델 로드
keywords_model = BertModel.from_pretrained('skt/kobert-base-v1')

# KeyBERT 모델 초기화
kw_model = KeyBERT(keywords_model)

# 파일 저장할 폴더 생성
# os.mkdir("/content/upload_file")
# os.mkdir("/content/image_file")

# PDF 라면 이미지로 변경
def pdf_to_png(files):
    path = f"/content/upload_file/{files}"
    doc = fitz.open(path)
    for i, page in enumerate(doc):
        img = page.get_pixmap()
        img.save(f"/content/image_file/{i}.png")

# OCR 함수
def detect_paragraphs(image_path):
    from google.cloud import vision
    # API키 가져오기
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/content/drive/MyDrive/api-project-397750607032-5ddc025931cd.json"

    # API 가져오기
    client = vision.ImageAnnotatorClient()

    # 주석을 추가할 이미지 파일 이름
    file_name = os.path.abspath(image_path)

    # 이미지 로드
    with io.open(file_name, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # 이미지 OCR
    response = client.document_text_detection(image=image)

    # 이미지 OCR 텍스트 전문
    full_text = response.full_text_annotation.text

    # 이미지 OCR 후 결과 (bbox, word 등)
    pages = response.full_text_annotation.pages

    # 텍스트 주석
    # 참고 : https://cloud.google.com/dotnet/docs/reference/Google.Cloud.Vision.V1/latest/Google.Cloud.Vision.V1.TextAnnotation.Types.DetectedBreak.Types.BreakType
    # 참고 : https://googleapis.github.io/googleapis/java/grpc-google-cloud-vision-v1/0.1.5/apidocs/com/google/cloud/vision/v1/TextAnnotation.DetectedBreak.BreakType.html
    breaks = vision.TextAnnotation.DetectedBreak.BreakType

    paragraphs = []
    lines = []

    for page in pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                para = "★문단시작★"
                line = ""
                for word in paragraph.words:
                    for symbol in word.symbols:
                        line += symbol.text
                        # breaks.SPACE : 공백
                        if symbol.property.detected_break.type == breaks.SPACE:
                            line += ' '
                        # breaks.EOL_SURE_SPACE : 줄 바꿈
                        if symbol.property.detected_break.type == breaks.EOL_SURE_SPACE:
                            line += ' '
                            lines.append(line)
                            para += line
                            line = ''
                        # breaks.LINE_BREAK : 단락을 끝내는 줄바꿈
                        if symbol.property.detected_break.type == breaks.LINE_BREAK:
                            lines.append(line)
                            para += line
                            line = ''
                paragraphs.append(para)

    return full_text, paragraphs

def summarize_paragraphs(paragraphs):

    # 문단 리스트를 512 토큰으로 나누기
    # 입력문장 리스트 : paragraphs

    # 모델 입력 데이터 생성
    input_data = []

    # 요약 데이터 생성
    summary =[]

    # 현재까지의 토큰 수 초기화
    current_token_count = 0

    for paragraph in paragraphs:
        # '★문단시작★'를 제거하고 문단을 토큰화
        paragraph_tokens = summary_tokenizer(paragraph.replace('★문단시작★', ''), return_tensors='pt', add_special_tokens=True).input_ids

        # 현재 문단을 추가해도 512 토큰을 넘지 않으면 추가
        if current_token_count + len(paragraph_tokens[0]) <= 512:
            if not input_data:
                input_data.append(paragraph_tokens)
                current_token_count += len(paragraph_tokens[0])
            else:
                # 이미 문단이 추가되어 있는 경우 이어서 추가
                input_data[-1] = torch.cat((input_data[-1], paragraph_tokens), dim=-1)
                current_token_count += len(paragraph_tokens[0])
        else:
            # 256 토큰을 넘어가면 새로운 입력으로 시작
            input_data.append(paragraph_tokens)
            current_token_count = len(paragraph_tokens[0])

    for inp in input_data:
        output = summary_model.generate(inp, eos_token_id=1, max_length=512, num_beams=4)
        t_output = summary_tokenizer.decode(output[0], skip_special_tokens=True)
        summary.append(t_output)

    return summary

def fulltext_keywords(full_text):

    kiwi.analyze(full_text)

    # 키워드 추출
    keywords = kw_model.extract_keywords(full_text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)

    # 명사 키워드 추출 함수 정의
    def extract_nouns(keyword_list):
        noun_keywords = []
        keywordscore = []
        for keyword, score in keywords:
            # 형태소 분석 수행
            tokens = kiwi.analyze(keyword)
            # print(tokens)
            for tk in tokens[0][0]:
                # print(tk)
                if tk.tag == "NNG":
                    if tk.form not in noun_keywords:
                        noun_keywords.append(tk.form)
                        keywordscore.append((tk.form,score))

        return noun_keywords, keywordscore
    noun_keywords, keywordscore = extract_nouns(keywords)
    return noun_keywords, keywordscore

def keyword_sum(noun_keyword, n):
    from collections import Counter
    # 키워드 리스트 하나로 합치기
    word_list = sum(noun_keyword,[])
    # 단어 빈도수 계산
    word_counts = Counter(word_list)

    # 가장 많이 중복된 단어 5개 선택
    top_n_words = [word for word, count in word_counts.most_common(n)]
    return top_n_words

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
# 코렙이라서 사용, flask에서는 다른 방법 필요
%cd /content/upload_file
from google.colab import files
uploaded = files.upload()

file_names = os.listdir('/content/upload_file')
if '.ipynb_checkpoints' in file_names:
    file_names.remove('.ipynb_checkpoints')

for files in file_names:
    if '.pdf' in files[-4:]:
	    pdf_to_png(files)

summarys, keywordscores, noun_keyword = [], [], []
full_text = ""
image_pathes = os.listdir('/content/image_file')
 # PDF 10장/ 2분 7초
for image_paths in image_pathes:
    image_path = f'/content/image_file/{image_paths}'
    text_full, paragraphs = detect_paragraphs(image_path)
    full_text = text_full
    summarys.append(summarize_paragraphs(paragraphs))
    noun_keywords, keywordscore = fulltext_keywords(full_text)
    keywordscores.append(keywordscore)
    noun_keyword.append(noun_keywords)
key_5 =  keyword_sum(noun_keyword, 5)

print(summarys, full_text, key_5)

In [22]:
import numpy as np
import fitz
import io
import os
from google.cloud import vision
from PIL import Image
from kiwipiepy import Kiwi
from transformers import BertModel
from keybert import KeyBERT
import torch
from transformers import PreTrainedTokenizerFast
from transformers.models.bart import BartForConditionalGeneration

# Kiwi, BERT 모델 및 KeyBERT 모델 초기화
kiwi = Kiwi()
keywords_model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(keywords_model)

# 요약 모델 및 토크나이저 초기화
summary_model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/KoBART-summarization/kobart_summary')
summary_tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

# OCR 함수
def detect_paragraphs(image_path):
    from google.cloud import vision

    # API키 가져오기
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"/content/drive/MyDrive/api-project-397750607032-5ddc025931cd.json"

    # API 가져오기
    client = vision.ImageAnnotatorClient()

    # 주석을 추가할 이미지 파일 이름
    file_name = os.path.abspath(image_path)

    # 이미지 로드
    with io.open(file_name, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # 이미지 OCR
    response = client.document_text_detection(image=image)

    # 이미지 OCR 텍스트 전문
    full_text = response.full_text_annotation.text

    # 이미지 OCR 후 결과 (bbox, word 등)
    pages = response.full_text_annotation.pages

    # 텍스트 주석
    # 참고 : https://cloud.google.com/dotnet/docs/reference/Google.Cloud.Vision.V1/latest/Google.Cloud.Vision.V1.TextAnnotation.Types.DetectedBreak.Types.BreakType
    # 참고 : https://googleapis.github.io/googleapis/java/grpc-google-cloud-vision-v1/0.1.5/apidocs/com/google/cloud/vision/v1/TextAnnotation.DetectedBreak.BreakType.html
    breaks = vision.TextAnnotation.DetectedBreak.BreakType

    paragraphs = []
    lines = []

    for page in pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                para = "★문단시작★"
                line = ""
                for word in paragraph.words:
                    for symbol in word.symbols:
                        line += symbol.text
                        # breaks.SPACE : 공백
                        if symbol.property.detected_break.type == breaks.SPACE:
                            line += ' '
                        # breaks.EOL_SURE_SPACE : 줄 바꿈
                        if symbol.property.detected_break.type == breaks.EOL_SURE_SPACE:
                            line += ' '
                            lines.append(line)
                            para += line
                            line = ''
                        # breaks.LINE_BREAK : 단락을 끝내는 줄바꿈
                        if symbol.property.detected_break.type == breaks.LINE_BREAK:
                            lines.append(line)
                            para += line
                            line = ''
                paragraphs.append(para)

    return full_text, paragraphs

def summarize_paragraphs(paragraphs):

    # 문단 리스트를 512 토큰으로 나누기
    # 입력문장 리스트 : paragraphs

    # 모델 입력 데이터 생성
    input_data = []

    # 요약 데이터 생성
    summary =[]

    # 현재까지의 토큰 수 초기화
    current_token_count = 0

    for paragraph in paragraphs:
        # '★문단시작★'를 제거하고 문단을 토큰화
        paragraph_tokens = summary_tokenizer(paragraph.replace('★문단시작★', ''), return_tensors='pt', add_special_tokens=True).input_ids

        # 현재 문단을 추가해도 512 토큰을 넘지 않으면 추가
        if current_token_count + len(paragraph_tokens[0]) <= 512:
            if not input_data:
                input_data.append(paragraph_tokens)
                current_token_count += len(paragraph_tokens[0])
            else:
                # 이미 문단이 추가되어 있는 경우 이어서 추가
                input_data[-1] = torch.cat((input_data[-1], paragraph_tokens), dim=-1)
                current_token_count += len(paragraph_tokens[0])
        else:
            # 256 토큰을 넘어가면 새로운 입력으로 시작
            input_data.append(paragraph_tokens)
            current_token_count = len(paragraph_tokens[0])

    for inp in input_data:
        output = summary_model.generate(inp, eos_token_id=1, max_length=512, num_beams=4)
        t_output = summary_tokenizer.decode(output[0], skip_special_tokens=True)
        summary.append(t_output)

    return summary

def fulltext_keywords(full_text):
    kiwi.analyze(full_text)
    keywords = kw_model.extract_keywords(full_text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)
    # 명사 키워드 추출 함수 정의
    def extract_nouns(keyword_list):
        noun_keywords = []
        keywordscore = []
        for keyword, score in keywords:
            # 형태소 분석 수행
            tokens = kiwi.analyze(keyword)
            # print(tokens)
            for tk in tokens[0][0]:
                # print(tk)
                if tk.tag == "NNG":
                    if tk.form not in noun_keywords:
                        noun_keywords.append(tk.form)
                        keywordscore.append((tk.form,score))

        return noun_keywords, keywordscore

    noun_keywords, keywordscore = extract_nouns(keywords)
    return noun_keywords, keywordscore


def process_images_and_text(image_folder, n):
    summarys, keywordscores, noun_keyword = [], [], []
    full_text = ""
    image_paths = os.listdir(image_folder)

    for image_path in image_paths:
        image_path = os.path.join(image_folder, image_path)
        text_full, paragraphs = detect_paragraphs(image_path)
        full_text = text_full
        summarys.append(summarize_paragraphs(paragraphs))
        noun_keywords, keywordscore = fulltext_keywords(full_text)
        keywordscores.append(keywordscore)
        noun_keyword.append(noun_keywords)

        def keyword_sum(noun_keyword, n):
            from collections import Counter
            # 키워드 리스트 하나로 합치기
            word_list = sum(noun_keyword,[])
            # 단어 빈도수 계산
            word_counts = Counter(word_list)

            # 가장 많이 중복된 단어 5개 선택
            top_n_words = [word for word, count in word_counts.most_common(n)]
            return top_n_words
    top_n_words =  keyword_sum(noun_keyword, n)
    return summarys, full_text, top_n_words

def pdf_to_img(file_folder):
    file_paths = os.listdir(file_folder)
    for image_path in file_paths:
        # PDF면 이미지 파일로 변환
        if image_path[-4:] == '.pdf':
            doc = fitz.open(image_path)
            # PDF to img
            for i, page in enumerate(doc):
                img = page.get_pixmap()
                img.save(f"./{image_path[2:-4]}_{i+1:03d}.png")
            # PDF 파일 삭제
            os.remove(image_path)



You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [25]:
# 이미지 폴더를 사용하여 process_images_and_text 함수 호출
image_folder = '/content/test'
summarys, full_text, key_5 = process_images_and_text(image_folder, 5) # 10장에 4분

# 추가 처리 또는 필요에 따라 결과를 출력합니다.
print(summarys, full_text, key_5)

[['이날은 교원인 듯한 양복쟁이를 동광학교에 태워다 주기로 하는 등 오래간만에 김첨지에게 닥친 운수 좋은 날이었다.', '김첨지는 오래간만에 김첨지에게 의지하면 오라질 년이 천방지축으로 냄비는에 손을 대고 끓였고 오래간만에 김첨지는 김첨지를 쉽게 물리쳤다.']] 운수 좋은날
현진건
새침하게 흐린 품이 눈이 올 듯하더니 눈은 아니 오고 얼다가 만 비가 추
적추적 내리는 날이었다.
이날이야말로 동소문 안에서 인력거꾼 노릇을 하는 김첨지에게는 오래간만
에도 닥친 운수 좋은 날이었다. 문안에 거기도 문밖은 아니지만 들어간답
시는 앞집 마마님을 전찻길까지 모셔다 드린 것을 비롯으로 행여나 손님이
있을까 하고 정류장에서 어정어정하며 내리는 사람 하나하나에게 거의 비는
듯한 눈결을 보내고 있다가 마침내 교원인 듯한 양복쟁이를 동광학교(東光
學校)까지 태워다 주기로 되었다.
첫 번에 삼십전, 둘째 번에 오십전 - 아침 댓바람에 그리 흉치 않은 일이
었다. 그야말로 재수가 옴불어서 근 열흘 동안 돈 구경도 못한 김첨지는 십
전짜리 백동화 서 푼, 또는 다섯 푼이 찰깍 하고 손바닥에 떨어질 제 거의
눈물을 흘릴 만큼 기뻤었다. 더구나 이날 이때에 이 팔십 전이라는 돈이 그
에게 얼마나 유용한지 몰랐다. 컬컬한 목에 모주 한 잔도 적실 수 있거니와
그보다도 앓는 아내에게 설렁탕 한 그릇도 사다 줄 수 있음이다.
그의 아내가 기침으로 쿨룩거리기는 벌써 달포가 넘었다. 조밥도 굶기를
먹다시피 하는 형편이니 물론 약 한 첩 써본 일이 없다. 구태여 쓰려면 못
쓸 바도 아니로되 그는 병이란 놈에게 약을 주어 보내면 재미를 붙여서 자
꾸 온다는 자기의 신조(信)에 어디까지 충실하였다. 따라서 의사에게 보
인 적이 없으니 무슨 병인지는 알 수 없으되 반듯이 누워 가지고 일어나기
는 새로 모로도 못 눕는 걸 보면 중증은 중증인 듯. 병이 이대도록 심해지
기는 열흘전에 조밥을 먹고 체한 때문이다. 그때도 김첨지가 오래간만에 돈
을 얻어서 좁쌀 한 뇌와 십 전짜리 나무 한 단을 사다 주었더니 